In [2]:
import pandas as pd
import numpy as np
import requests
import json
import gviz_api
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('display.max_columns', None)   

In [ ]:
#     engine = get_db()
#     sql = """
#     SELECT dublinbikedb.static.number, available_bikes, last_update
#     FROM dublinbikedb.static 
#     JOIN dublinbikedb.dynamic ON dublinbikedb.static.number = dublinbikedb.dynamic.number 
#     """

# For feeding SQL data to jupyter notebook 2.
#     chart = engine.execute(sql)
#     chartJson = []
#     for i in chart:
#         chartJson.append(dict(i))

#     return chartJson.to_json()

In [10]:
chart1 = requests.get("http://localhost:5000/chart/1")

In [11]:
myJson = json.loads(chart1.text)

In [15]:
df = json_normalize(myJson, sep='_')
df.head(5)

,available_bikes,last_update,number
0,7,1.520865e+12,42
1,7,1.520865e+12,42
2,17,1.521136e+12,42
3,22,1.521137e+12,42
4,25,1.521139e+12,42


In [16]:
df['last_update'] =  pd.to_datetime((df['last_update']//1000), unit='s')

In [19]:
df.head(5)

,available_bikes,last_update,number
0,7,2018-03-12 14:23:47,42
1,7,2018-03-12 14:30:30,42
2,17,2018-03-15 17:48:57,42
3,22,2018-03-15 18:10:19,42
4,25,2018-03-15 18:33:13,42


In [25]:
df.set_index(df["last_update"],inplace=True, drop=True)

In [26]:
df.head(5)

,available_bikes,last_update,number
last_update,,,
2018-03-12 14:23:47,7,2018-03-12 14:23:47,42
2018-03-12 14:30:30,7,2018-03-12 14:30:30,42
2018-03-15 17:48:57,17,2018-03-15 17:48:57,42
2018-03-15 18:10:19,22,2018-03-15 18:10:19,42
2018-03-15 18:33:13,25,2018-03-15 18:33:13,42


In [28]:
#df.drop(['last_update'], axis=1).head(5)

,available_bikes,number
last_update,,
2018-03-12 14:23:47,7,42
2018-03-12 14:30:30,7,42
2018-03-15 17:48:57,17,42
2018-03-15 18:10:19,22,42
2018-03-15 18:33:13,25,42


In [49]:
group = pd.DataFrame(df.groupby([df.number, df.index.dayofweek, df.index.hour]).mean().unstack(level=1))

In [50]:
group.head(5)

available_bikes                                          \
last_update                    0.0       1.0       2.0       3.0       4.0   
number last_update                                                           
1      0.0                6.720000  1.852459  0.523810  0.451613  3.200000   
       1.0                7.280000  1.984375  0.584615  0.437500  3.245902   
       2.0                7.000000  1.875000  0.562500  0.444444  3.147541   
       3.0                6.844444  1.950000  0.557377  0.566667  3.087719   
       4.0                7.833333  2.933333  1.241935  1.281250  4.606557   

                                          
last_update               5.0        6.0  
number last_update                        
1      0.0          11.350877  22.604167  
       1.0          10.741935  22.250000  
       2.0          10.915254  22.367347  
       3.0          10.610169  22.065217  
       4.0          11.885246  22.816327

In [51]:
group.columns.set_levels(["Monday", "Tuesday", "Wednesday", 
                          "Thursday", "Friday", "Saturday", "Sunday"],level=1,inplace=True)

In [197]:
group.head(5)

available_bikes                                          \
last_update                 Monday   Tuesday Wednesday  Thursday    Friday   
number last_update                                                           
1      0.0                6.720000  1.852459  0.523810  0.451613  3.200000   
       1.0                7.280000  1.984375  0.584615  0.437500  3.245902   
       2.0                7.000000  1.875000  0.562500  0.444444  3.147541   
       3.0                6.844444  1.950000  0.557377  0.566667  3.087719   
       4.0                7.833333  2.933333  1.241935  1.281250  4.606557   

                                          
last_update          Saturday     Sunday  
number last_update                        
1      0.0          11.350877  22.604167  
       1.0          10.741935  22.250000  
       2.0          10.915254  22.367347  
       3.0          10.610169  22.065217  
       4.0          11.885246  22.816327

In [198]:
group.columns

MultiIndex(levels=[['available_bikes'], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']],
           labels=[[0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6]],
           names=[None, 'last_update'])

In [199]:
test_group = group.head(48)

In [200]:
test_group.head(3)

available_bikes                                          \
last_update                 Monday   Tuesday Wednesday  Thursday    Friday   
number last_update                                                           
1      0.0                    6.72  1.852459  0.523810  0.451613  3.200000   
       1.0                    7.28  1.984375  0.584615  0.437500  3.245902   
       2.0                    7.00  1.875000  0.562500  0.444444  3.147541   

                                          
last_update          Saturday     Sunday  
number last_update                        
1      0.0          11.350877  22.604167  
       1.0          10.741935  22.250000  
       2.0          10.915254  22.367347

In [201]:
test_group.tail(3)

available_bikes                                             \
last_update                 Monday    Tuesday  Wednesday   Thursday    Friday   
number last_update                                                              
2      21.0               7.737705  12.903226  15.651515  10.616667  5.068966   
       22.0               7.187500  14.546875  16.650000  11.883333  8.241935   
       23.0               8.016667  16.049180  17.491803  12.315789  9.267857   

                                        
last_update         Saturday    Sunday  
number last_update                      
2      21.0         7.051282  6.533333  
       22.0         6.428571  9.444444  
       23.0         6.708333  9.900000

In [202]:
test_group.columns

MultiIndex(levels=[['available_bikes'], ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']],
           labels=[[0, 0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5, 6]],
           names=[None, 'last_update'])

In [203]:
for number, new_df in test_group.groupby(level=0):
    print('Number: ', number)
    print("hour: ", new_df['available_bikes']['Monday'].reset_index(drop=True).to_json())

Number:  1
hour:  {"0":6.72,"1":7.28,"2":7.0,"3":6.8444444444,"4":7.8333333333,"5":9.7959183673,"6":10.0638297872,"7":14.3,"8":21.5957446809,"9":22.7872340426,"10":23.8958333333,"11":20.6444444444,"12":21.12,"13":16.7551020408,"14":18.5862068966,"15":13.6307692308,"16":11.5483870968,"17":10.0158730159,"18":7.9411764706,"19":6.9473684211,"20":3.8125,"21":3.5,"22":3.2615384615,"23":2.2131147541}
Number:  2
hour:  {"0":9.7021276596,"1":10.62,"2":10.1632653061,"3":10.0697674419,"4":9.7916666667,"5":7.5714285714,"6":5.4583333333,"7":2.3333333333,"8":0.4680851064,"9":2.1914893617,"10":4.3958333333,"11":3.7755102041,"12":3.9130434783,"13":5.3725490196,"14":4.5737704918,"15":4.9032258065,"16":4.09375,"17":4.15,"18":5.921875,"19":6.4603174603,"20":7.0625,"21":7.737704918,"22":7.1875,"23":8.0166666667}


In [204]:
for i in test_group['available_bikes'].columns:
    print(i)

Monday
Tuesday
Wednesday
Thursday
Friday
Saturday
Sunday


In [205]:
groupJson = {}
for number, new_df in test_group.groupby(level=0):
    groupJson[str(number)] = {}
    for day in test_group['available_bikes'].columns:
        groupJson[str(number)][str(day)] = new_df['available_bikes'][day].reset_index(drop=True).tolist()
print(groupJson)

{'1': {'Monday': [6.72, 7.28, 7.0, 6.844444444444444, 7.833333333333333, 9.795918367346939, 10.063829787234043, 14.3, 21.595744680851062, 22.78723404255319, 23.895833333333332, 20.644444444444446, 21.12, 16.755102040816325, 18.586206896551722, 13.63076923076923, 11.548387096774194, 10.015873015873016, 7.9411764705882355, 6.947368421052632, 3.8125, 3.5, 3.2615384615384615, 2.2131147540983607], 'Tuesday': [1.8524590163934427, 1.984375, 1.875, 1.95, 2.933333333333333, 6.193548387096774, 11.80327868852459, 14.955223880597014, 20.095238095238095, 18.184615384615384, 20.3125, 19.95, 14.523809523809524, 17.921875, 15.564516129032258, 15.133333333333333, 12.063492063492063, 11.508474576271187, 9.26984126984127, 6.953125, 2.6865671641791047, 1.3384615384615384, 0.22950819672131148, 0.6], 'Wednesday': [0.5238095238095238, 0.5846153846153846, 0.5625, 0.5573770491803278, 1.2419354838709677, 4.015384615384615, 9.82258064516129, 10.841269841269842, 12.88888888888889, 13.967213114754099, 15.661290322

In [206]:
json.dumps(groupJson)

'{"1": {"Monday": [6.72, 7.28, 7.0, 6.844444444444444, 7.833333333333333, 9.795918367346939, 10.063829787234043, 14.3, 21.595744680851062, 22.78723404255319, 23.895833333333332, 20.644444444444446, 21.12, 16.755102040816325, 18.586206896551722, 13.63076923076923, 11.548387096774194, 10.015873015873016, 7.9411764705882355, 6.947368421052632, 3.8125, 3.5, 3.2615384615384615, 2.2131147540983607], "Tuesday": [1.8524590163934427, 1.984375, 1.875, 1.95, 2.933333333333333, 6.193548387096774, 11.80327868852459, 14.955223880597014, 20.095238095238095, 18.184615384615384, 20.3125, 19.95, 14.523809523809524, 17.921875, 15.564516129032258, 15.133333333333333, 12.063492063492063, 11.508474576271187, 9.26984126984127, 6.953125, 2.6865671641791047, 1.3384615384615384, 0.22950819672131148, 0.6], "Wednesday": [0.5238095238095238, 0.5846153846153846, 0.5625, 0.5573770491803278, 1.2419354838709677, 4.015384615384615, 9.82258064516129, 10.841269841269842, 12.88888888888889, 13.967213114754099, 15.66129032